# Semantic Segmentation with FastSCNN - changing input dimensions

The purpose of this model is to train the network on the train dataset and then export the model in onnx format and also save the state_dict of the pytorch model for later inference

**Do not run this cell outside of Google Colaboratory**

In [ ]:
!pip3 install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52
!pip install torchmetrics
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/xolotl18/Master_Thesis
!pip install onnx
!pip install onnxruntime

## Imports

In [1]:
import copy
import random
import os
import sys
import shutil
from urllib.request import urlretrieve

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import JaccardIndex
from statistics import mean
import torchvision.transforms as T
import torch.onnx
import onnx
import onnxruntime
from torchsummary import summary


from utils.lr_scheduler import PolynomialLRDecay
from models.experiments.fast_scnn_mod import FastSCNN as fastscnn_mod

from utils.dataset import PackagesDataset, PackagesInferenceDataset
from utils.evaluation import Evaluate
cudnn.benchmark = True

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

params = {
    "device" : device,
    "lr" : 0.01,
    "batch_size" : 8,
    "num_workers" : 4,
    "epochs" : 400,
}
print(f"The device is : {device}")

The device is : cuda


In [3]:
def train(train_loader, model, criterion, optimizer, scheduler, epoch, params):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        optimizer.zero_grad()

        images, targets = data
        images = images.to(params["device"], non_blocking=True)
        targets = targets.to(params["device"], non_blocking=True)

        outputs = model(images)
        targets = torch.unsqueeze(targets, 1)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step(epoch)
        running_loss += loss.item()*images.size(0)

def validate(val_loader, model, criterion, epoch, params):
    model.eval()
    running_loss=0.0
    with torch.no_grad():
        for i, data in enumerate(val_loader, start=1):
                images, targets = data
                images = images.to(params["device"], non_blocking=True)
                targets = targets.to(params["device"], non_blocking=True)
                output = model(images).squeeze(1)
                loss = criterion(output, targets)
                running_loss += loss.item()*images.size(0)
    return running_loss

In [4]:
def predict(model, params, test_dataset):
    test_loader = DataLoader(
        test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
    )
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, masks, (original_heights, original_widths) in test_loader:
            images = images.to(params["device"], non_blocking=True)
            output = model(images)
            probabilities = torch.sigmoid(output.squeeze(1))
            predicted_masks = (probabilities >= 0.5).float() * 1
            predicted_masks = predicted_masks.cpu().numpy()
            for predicted_mask, gt, original_height, original_width in zip(
                predicted_masks, masks.numpy(), original_heights.numpy(), original_widths.numpy()
            ):
                predictions.append((predicted_mask, gt, original_height, original_width))
    return predictions

In [5]:
def remove_initializer_from_input(model_path):

    onnxmodel = onnx.load(model_path+".onnx")
    if onnxmodel.ir_version < 4:
        print("Model with ir_version below 4 requires to include initilizer in graph input")
        return

    inputs = onnxmodel.graph.input
    name_to_input = {}
    for input in inputs:
        name_to_input[input.name] = input

    for initializer in onnxmodel.graph.initializer:
        if initializer.name in name_to_input:
            inputs.remove(name_to_input[initializer.name])
    out_path = model_path+"_noinit.onnx"
    onnx.save(onnxmodel, out_path)

## Load image and label files into Dataset objects

The dataset has already been divided into train, validation and test folders in the notebook **Desktop/Master_Thesis/preparation/dataset_traintest_split.ipynb**

In [6]:
c_dir = os.getcwd()
dataset_directory = os.path.join(c_dir, "full_dataset")

train_images_directory = os.path.join(dataset_directory, "train/images")
train_masks_directory = os.path.join(dataset_directory, "train/labels")
val_images_directory = os.path.join(dataset_directory, "val/images")
val_masks_directory = os.path.join(dataset_directory, "val/labels")
test_images_directory = os.path.join(dataset_directory, "test/images")
test_masks_directory = os.path.join(dataset_directory, "test/labels")

#make sure that image_filenames only contains png files
train_images_filenames = []
train_images_filenames = [ item for item in os.listdir(train_images_directory) if item.endswith(".png") ]
val_images_filenames = []
val_images_filenames = [ item for item in os.listdir(val_images_directory) if item.endswith(".png") ]
test_images_filenames = []
test_images_filenames = [ item for item in os.listdir(test_images_directory) if item.endswith(".png") ]

for names_list, split in zip((train_images_filenames, val_images_filenames, test_images_filenames), ('train', 'validation', 'test')):
  print(f"The size of the {split} set is : {len(names_list)}")
  print()

The size of the train set is : 300

The size of the validation set is : 40

The size of the test set is : 60



In [7]:
dimension_multipliers =[1.0, 0.5, 0.4375, 0.375, 0.3125, 0.25, 0.1875]

In [8]:
for m in dimension_multipliers:
    train_transform = A.Compose(
        [
            A.Resize(int(m*540), int(m*960)),
            A.RandomCrop(int(m*512), int(m*512)),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),
            A.RandomBrightnessContrast(p=0.3),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )
    val_transform = A.Compose(
        [
            A.Resize(int(m*540), int(m*960)),
            A.CenterCrop(int(m*512), int(m*512)),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )
    test_transform = A.Compose(
        [
            A.Resize(int(m*512), int(m*512)),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )

    train_dataset = PackagesDataset(train_images_filenames, train_images_directory, 
                                    train_masks_directory, transform=train_transform,)
    val_dataset = PackagesDataset(val_images_filenames, val_images_directory,
                                  val_masks_directory, transform=val_transform,)
    test_dataset = PackagesInferenceDataset(test_images_filenames, test_images_directory, 
                                            test_masks_directory, transform=test_transform,)
    input_sample = train_dataset[0][0].numpy()
    chans, dim1, dim2 = input_sample.shape
    print()
    print(f"Input of dimension {dim1}x{dim2}x{chans}")
    print()
    #we run the baseline model without pyramid pooling since we have affirmed that its effect is 
    #irrelevant on the dataset in use and it causes issues when varying input size
    model = fastscnn_mod(in_channels=3, num_classes=1, a=1.0, t=6, r=3, pp=False).to(params["device"])
    
    train_loader = DataLoader(
    train_dataset,
    batch_size=params["batch_size"],
    shuffle=True,
    num_workers=params["num_workers"],
    pin_memory=False,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=params["batch_size"],
        shuffle=True,
        num_workers=params["num_workers"],
        pin_memory=False,
    )


    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
    scheduler = PolynomialLRDecay(optimizer, max_decay_steps=params["epochs"], end_learning_rate=0.0001, power=0.9)

    best_loss = sys.float_info.max
    model_ckpt = copy.deepcopy(model.state_dict())
    for epoch in tqdm(range(1, params["epochs"]+1)):
        train(train_loader, model, criterion, optimizer, scheduler, epoch, params)
        epoch_loss = validate(val_loader, model, criterion, epoch, params)
        #select the best model based on the loss on the validation set
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            model_ckpt = copy.deepcopy(model.state_dict())

    model.load_state_dict(model_ckpt)    
    predictions = predict(model, params, test_dataset)

    evaluator = Evaluate(predictions)
    iou, dice = evaluator.get_metrics().values()
    print(f"The Intersection over Union score is : {iou:.4f}")
    print(f"The Dice Coefficient is : {dice:.4f}")
    
    #select the name that the model will be saved with
    model_name = "fastscnn_"+str(dim1)
    model_path = os.path.join(c_dir, "model_checkpoints/experiments/input_dim", model_name)
    torch.save(model.state_dict(), model_path+".pt")
    
    #put the model in inference mode
    model.eval()
    #generate dummy input for onnx export
    x = torch.randn(1, 3, int(m*512), int(m*512), requires_grad=True).cuda()
    torch_out = model(x)

    # Export the model
    torch.onnx.export(model,                         # model being run
                      x,                             # model input (or a tuple for multiple inputs)
                      model_path+".onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,            # store the trained parameter weights inside the model file
                      opset_version=11,              # the ONNX version to export the model to
                      do_constant_folding=True,      # whether to execute constant folding for optimization
                      input_names = ['input'],       # the model's input names
                      output_names = ['output'],     # the model's output names
                      operator_export_type=torch.onnx.OperatorExportTypes.ONNX,
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})
    remove_initializer_from_input(model_path)
    print("Onnx model saved")
    print()


Input of dimension 512x512x3

The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  0


  0%|          | 0/400 [00:00<?, ?it/s]

The Intersection over Union score is : 0.9775
The Dice Coefficient is : 0.9886
Onnx model saved


Input of dimension 256x256x3

The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  0


  0%|          | 0/400 [00:00<?, ?it/s]

The Intersection over Union score is : 0.9706
The Dice Coefficient is : 0.9851
Onnx model saved


Input of dimension 224x224x3

The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  0


...

The Intersection over Union score is : 0.9621
The Dice Coefficient is : 0.9807
Onnx model saved


Input of dimension 128x128x3

The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  0


  0%|          | 0/400 [00:00<?, ?it/s]

The Intersection over Union score is : 0.9537
The Dice Coefficient is : 0.9763
Onnx model saved


Input of dimension 96x96x3

The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  0


  0%|          | 0/400 [00:00<?, ?it/s]

The Intersection over Union score is : 0.9357
The Dice Coefficient is : 0.9667
Onnx model saved

